### Introduction

In the following notebook, I will be cleaning a raw data file of listings data from Inside Airbnb

**Read in libraries**

In [44]:
import numpy as np
import pandas as pd
import swifter
import matplotlib.pyplot as plt
import seaborn as sns

**Set notebook preferences**

In [45]:
#Set pandas preferences
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

**Read in data**

In [ ]:
#Set path to data on local machine
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\01_Raw\SF Airbnb'

#Read in data
df = pd.read_csv(path + '/2020_0519_Aggregated_Listings.csv',parse_dates= ['host_since','last_review', 'first_review'],index_col=0)

### Data Overview

**Preview Data**

In [ ]:
#Display data, print shape
print('Data shape:', df.shape)
display(df.head(3))

**View data description**

In [ ]:
#View data description
df.describe().T

## Data Cleaning

### Drop Columns

**Drop mostly homogenous/redundant columns and columns with only missing values**

In [ ]:
#Extract cols with values with more than 1 unique value
df = df.loc[:,(df.nunique() != 1)]

In [ ]:
#Drop missing columns
df.dropna(axis =1,how = 'all', inplace = True)

#Drop redundant columns
df.drop(['jurisdiction_names', 'market','state','neighbourhood','street','smart_location','host_neighbourhood'], axis = 1, inplace = True)

In [ ]:
#Inspect cols with <=2 unique values
inspect = df.loc[:, (df.nunique() <=2)].columns.to_list()

#Check
display(df[inspect].head(3))

In [ ]:
#Create dictionary for mapping
mapping = {'t':1,'f':0}

#Map 1's and 0's on t's and f's
df[inspect] = df[inspect].apply(lambda x: x.map(mapping, na_action='ignore'))

#Check
display(df[inspect].head(3))

**Drop columns containing url data or pertain to webscraping**

In [ ]:
#Subset column headers containing 'url' or 'scrape' and store in drop
drop = list(df.filter(regex='url|scrape').columns)

#Drop drop list and check
df.drop(columns= df[drop], inplace=True)
df.head(1)

## Check for high correlations between features

**Prepare data**

In [ ]:
#Create correlation matrix and capture absolute values of correlations
c = df.corr().abs()

#Create a df that stores correlations between features >.9
s = c.unstack()
so = s.sort_values(kind="quicksort").reset_index()
so.columns = ['feat1','feat2','corr']
so = so.loc[ (so.feat1 != so.feat2 )& (so['corr'] > .9)]

#Capture list of features
feats =so.feat1.unique()

#Subset df by cols in feats and create corr
corr= df[feats].corr()

**Create heatmap**

In [ ]:
#Create fig
f, ax = plt.subplots(figsize = (13,13))

#Plot corr as heat map
sns.heatmap(data = corr, annot=True,fmt='.1%', cmap = 'coolwarm', ax=ax,
            linewidths=1.0, square=1);

**Drop cols with high collinearity**

In [ ]:
#Cols with high collinearity
drop = ['calculated_host_listings_count_entire_homes','maximum_nights_avg_ntm', 'maximum_maximum_nights','minimum_maximum_nights',
        'maximum_minimum_nights','minimum_minimum_nights', 'minimum_nights_avg_ntm', 'host_total_listings_count']

#Drop drop
df.drop(drop, axis=1, inplace = True)

## Clean up object and numeric columns

**Clean up numeric columns**

In [ ]:
#Filter cols pertaining to prices and assign col names as a list to money_cols
money_cols = df.filter(regex = 'people|deposit|price|fee$|rate').columns.tolist()

#Remove $, and set type as numeric for money_cols
df[money_cols] = df[money_cols].replace('[$|,|%]','',regex = True).astype('float')

#Check
display(df[money_cols].head(3))

**Clean up object columns**

In [ ]:
#Create list of columns to apply cleaning to
objects = df.select_dtypes('object').columns.to_list()

#Check
display(df[objects].head(3))

In [ ]:
#Remove quotes and brackets. Keeping other punctuation
df[objects] = df[objects].apply(lambda x : x.str.replace('[{"}_]',' '))

#Check
display(df[objects].head())

### Missing Data

#### Datetime64

In [ ]:
#Import missing_calculator
from Missing_Stats import missing_calculator

#View missing statistics - datetime64
display(missing_calculator(df, data_type='datetime64'))

Because there is no inherent order within the data and most of the review data is captured, we will leave datetime data as is

#### Float64

In [ ]:
#View missing statistics - float64
display(missing_calculator(df, data_type='float64'))

**Drop columns w/ mostly missing values**

Leaving weekly and monthly price since not all hosts have to offer the service and they are sometimes at a special rate compared to the per night rental.

Keeping reviews_per_month as is for now, may fill with mean if needed.

In [ ]:
#Dropping square_feet and host_acceptance_rate
df.drop(['square_feet','host_acceptance_rate'], axis = 1, inplace = True)

**Convert review scores to categories**

In [ ]:
#Get list of cols pertaining to review scores
review_cols = df.filter(regex='review_scores').columns.tolist()

#Fill review_cols missing values with 'No review data' and convert into category
df[review_cols] = df[review_cols].fillna('No review data')
df[review_cols] = df[review_cols].astype('category')

**Fill remaining cols**

In [ ]:
#Subset cols to apply nafill w/ median
cols = ['security_deposit', 'host_response_rate', 'cleaning_fee']

#Fill na w/ median
df[cols]=df[cols].fillna(df.median().iloc[0])

#Subset cols to apply nafill w/ mode
cols = ['beds', 'bathrooms', 'bedrooms']

#Fill na w/mode
df[cols] = df[cols].fillna(df.mode().iloc[0])

#Capture cols for filling
cols = ['host_has_profile_pic', 'host_identity_verified', 'host_is_superhost', 'host_listings_count' ]

#Ffill, bfill remaining missing floats
df.loc[:,cols]=df.loc[:,cols].ffill().bfill()

#Check
display(missing_calculator(df, data_type='float64'))

#### Objects

In [ ]:
#Store missins stats in missin gdf
missing = missing_calculator(df, data_type='object')

#View missing statistics - object
display(missing)

Leaving the follwing text columns alone because they reflect non-mandatory inputs from the host about the living space they are renting out. Likely will not be used for later modeling:

* notes
* access
* interactiion
* transit
* house_rules, 
* neighborhood_overview 
* host_about 
* space
* host_location
* host_name
* host_neighbourhood

**Miscellaneous Column cleaning**

In [ ]:
#Cols for ffill and bfill
cols = ['license','host_response_time','cancellation_policy']

#Ffill, bfill objects
df.loc[:,cols]=df.loc[:,cols].ffill().bfill()

df.groupby('is_location_exact')['is_location_exact'].count()

**City clean-up**

In [ ]:
#View unique values in city
print('Unique values in city:\n', df.city.unique())

#Fix Daly City, San Francisco
df.city.replace('[^A-Za-z0-9\s]','San Francisco',regex = True, inplace=True)
df.city.replace('Da.*','Daly City',regex = True, inplace=True)
df.city.replace('(San\sF|Noe|B|Nor).*','San Francisco',regex = True, inplace=True)

#Fill na with San Francisco
df.city.fillna('San Francisco', inplace=True)

#Check
print('Unique values in city:\n', df.city.unique())

**Zipcode clean-up**

In [ ]:
#View unique values in zipcode
print('Unique values in zipcode:\n', df.zipcode.unique())

#Remove white spaces and CA
df.zipcode.replace('[CA]*\s*','', regex = True, inplace=True)

#Read in libraries to resolve missing values for zipcode using lat/long
from uszipcode import SearchEngine
from uszipcode import Zipcode

#Instantiate SearchEngine
zipsearch = SearchEngine(simple_zipcode=True)

#Write function that finds zip given lat and long data
def get_zipcode(lat, lon):
    result = zipsearch.by_coordinates(lat = lat, lng = lon, returns = 1)
    return result[0].zipcode

#Apply get_zipcode and assign to Zipcode
df.zipcode[df.zipcode.isna()]= df[df.zipcode.isna()][['latitude', 'longitude']].swifter.apply(lambda x: get_zipcode(x.latitude, x.longitude), axis =1)

#Check
print('Unique values in zipcode:\n', df.zipcode.unique())

**Final Check Missing Values**

In [ ]:
display(missing_calculator(df))

### Write to csv

In [ ]:
#Print final shape of df
print('Shape of cleaned data:', df.shape)

#Set path to local machine
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python\In Progress\Airbnb - San Francisco\Data\02_Cleaned'

#Write file
df.to_csv(path + '/2020_0520_Listings_Cleaned.csv')